In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nhl_correlation(): 
    
    cities = pd.read_html("assets/wikipedia_data.html")
    cities = cities[1] #Gets the second table of the page
    #Let's make our data easier to manage 
    cities.rename(columns = {"Population (2016 est.)[8]":"Population", "Metropolitan area":"City"}, inplace = True)
    #We keep only the important stuff
    cities_nhl = cities[["City","Population","NHL"]]
    #Basic data clean up process
    cities_nhl.loc[:,"NHL"] = cities_nhl.loc[:,"NHL"].replace("—",np.nan)
    #We want to replace everything in [] with ""
    cities_nhl.loc[:,"NHL"] = cities_nhl.loc[:,"NHL"].replace("\[.*\]","",regex = True)
    #We will drop all our NaN rows cause we just don't need them 
    cities_nhl = cities_nhl.dropna()
    #We will turn them into NaN values and just drop them [the cities that don't have NHL teams]
    cities_nhl.loc[:,"NHL"] = cities_nhl.loc[:,"NHL"].replace("",np.nan)
    cities_nhl = cities_nhl.dropna()
    cities_nhl = cities_nhl[:-1]
    #----------Now we work on the other table------------------
    nhl_table = pd.read_csv("assets/nhl.csv")
    nhl_table = nhl_table.drop(0)
    #We keep only the 2018 data
    mask2018 = nhl_table["year"] == 2018
    nhl_table = nhl_table[mask2018]
    nhl_table = nhl_table.drop([9,18,26]) #We drop the columns that have info about divisions
    nhl_table["team"] = nhl_table["team"].str.replace("*","") #We use the .str module because that is what works with string parsing and it gets rid of the *
    #We want to make a W/L ratio but for some reason our data in W and L is in string format 
    nhl_table["W"] = nhl_table["W"].astype(float)
    nhl_table["L"] = nhl_table["L"].astype(float)
    nhl_table["W/L"] = nhl_table["W"]/(nhl_table["L"]+nhl_table["W"])
    #This is the function that will find the problematic team names 
    easy_named_teams = []
    problematic_names = []

    #This function can check if the city of the team in the nhl_stats is found in 
    #our cities column
    
    metro_rules = cities_nhl
    nhl_stats = nhl_table
    
    def assign_city(tname): 
        if tname.split()[0] in str(metro_rules["City"].unique()):
            easy_named_teams.append(tname)
        else: 
            problematic_names.append(tname)
    
    nhl_stats["team"].apply(lambda x: assign_city(x))
    nhl_stats["City"] = np.nan
    
    #We will make a mapping dictionary 
    problematic_mapping = {
        "Florida Panthers": "Miami–Fort Lauderdale",
        "Carolina Hurricanes": "Raleigh", 
        "Minnesota Wild": "Minneapolis–Saint Paul",
        "Colorado Avalanache": "Denver",
        "Arizona Coyotes": "Phoenix", 
        "Anaheim Ducks": "Los Angeles"
    }
    
    def team_rename(row, dictionary): 
        team_name = row["team"] #Get the team name from the team column 
        if team_name in dictionary.keys(): #If it's in that mapping add the value in the City col
            row["City"] = dictionary[team_name]
        return row

    
    #We will now apply this with the dictionary = problematic_mapping 
    nhl_stats = nhl_stats.apply(team_rename, axis = 1, dictionary = problematic_mapping)
    nhl_stats.loc[22,"City"] = "Denver"
    
    def find_city_by_team(tname):
        result = metro_rules[metro_rules["City"].str.startswith(tname.split()[0])]
        return result["City"].iloc[0]

    easy_named_teams_dict = {}

    for team in easy_named_teams: 
        try: easy_named_teams_dict[team] = find_city_by_team(team)
        except: continue
    
    
    nhl_stats = nhl_stats.apply(team_rename, axis = 1, dictionary = easy_named_teams_dict)
    nhl_stats.loc[27,"City"] = "Las Vegas"
    
    mean_wl = nhl_stats["W/L"].groupby(nhl_stats["City"]).mean()
    #We merge our metro rules with the mean_wl to have the common city pops and w/l
    final = pd.merge(metro_rules, mean_wl, how = "inner", on = "City")
    
    correlation = stats.pearsonr(final["Population"].astype(float), final["W/L"].astype(float))
    
    return correlation[0]

: 